# Общая информация
__Цель:__ сделать fine-tuning mGPT by Sber

__Задачи:__
1) Выбрать вопросы с определенным тегом, например python

2) Понять формат входных и выходных данных, например перед вопросов, возможно надо ставить [QUESTION]

3) Сделать torch Dataset

4) Определить, как делать evaluation

5) Способ трэкинга

6) Проанализировать результаты

# Импорт библиотек

In [3]:
# from google.colab import drive

# drive.mount("/content/drive")

In [2]:
#!pip install transformers

In [3]:
from google.colab import drive

drive.mount("/content/drive")

import pickle

import numpy as np
import pandas as pd
import torch
import wandb
from datasets import load_metric
from torch.utils.data import Dataset, random_split
from tqdm import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          IntervalStrategy, Trainer, TrainingArguments)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
# зафиксируем random seed
torch.manual_seed(42)

# Чтение данных

In [8]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vkr_data/filtered_df.csv")

In [6]:
pd.set_option("display.max_colwidth", None)

In [48]:
# pd.set_option("display.max_colwidth", None)
# with open( f'../../data/filtered_df.p', 'rb') as f:
#     df = pickle.load(f)

Отсортируем датасет по времени и разобьем его на train test

In [9]:
df = df.sort_values("Q_Date")

In [10]:
df = df.loc[df.apply(lambda x: f"python" in x.Tag, axis=1)]

In [11]:
train_df, test_df = np.split(df, [int(0.75 * len(df))])

# Загрузим модель

In [12]:
device = torch.device("cuda") if torch.cuda.is_available else "cpu"

In [13]:
model_name = "EleutherAI/gpt-neo-1.3B"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="<|startoftext|>",
    eos_token="<|endoftext|>",
    pad_token="<|pad|>",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [16]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 2048)

In [58]:
for n, p in model.named_parameters():
    if "transformer.h" in n:
        layer_num = int(n.split(".")[2])
        if "ln_" not in n and layer_num > 0 and layer_num < 23:
            p.requires_grad = False
            print("Freeze", n)

Freeze transformer.h.1.attn.attention.k_proj.weight
Freeze transformer.h.1.attn.attention.v_proj.weight
Freeze transformer.h.1.attn.attention.q_proj.weight
Freeze transformer.h.1.attn.attention.out_proj.weight
Freeze transformer.h.1.attn.attention.out_proj.bias
Freeze transformer.h.1.mlp.c_fc.weight
Freeze transformer.h.1.mlp.c_fc.bias
Freeze transformer.h.1.mlp.c_proj.weight
Freeze transformer.h.1.mlp.c_proj.bias
Freeze transformer.h.2.attn.attention.k_proj.weight
Freeze transformer.h.2.attn.attention.v_proj.weight
Freeze transformer.h.2.attn.attention.q_proj.weight
Freeze transformer.h.2.attn.attention.out_proj.weight
Freeze transformer.h.2.attn.attention.out_proj.bias
Freeze transformer.h.2.mlp.c_fc.weight
Freeze transformer.h.2.mlp.c_fc.bias
Freeze transformer.h.2.mlp.c_proj.weight
Freeze transformer.h.2.mlp.c_proj.bias
Freeze transformer.h.3.attn.attention.k_proj.weight
Freeze transformer.h.3.attn.attention.v_proj.weight
Freeze transformer.h.3.attn.attention.q_proj.weight
Freeze t

Отберем только те строки, в тегах которых есть слово _python_

In [59]:
questions = df.Q_Body
max_length = max([len(tokenizer.encode(question)) for question in questions])
print(f"Max length: {max_length}")

NameError: ignored

In [18]:
class Q_A_Dataset(Dataset):
    def __init__(self, df, tokenizer, max_length, tag):
        df = df.loc[df.apply(lambda x: f"{tag}" in x.Tag, axis=1)]
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        self.answers = []
        self.questions = []

        for _, row in df.iterrows():
            prep_text = f"<|startoftext|>Question: {row.Q_Body}\nAnswer: {row.A_Body}<|endoftext|>"

            question_len = len(
                tokenizer(
                    f"<|startoftext|>Question: {row.Q_Body}\nAnswer:",
                )["input_ids"]
            )

            encoding_dict = tokenizer(
                prep_text, truncation=True, max_length=max_length, padding="max_length"
            )

            self.input_ids.append(torch.tensor(encoding_dict["input_ids"]))
            self.attn_masks.append(torch.tensor(encoding_dict["attention_mask"]))
            self.labels.append(torch.tensor(encoding_dict["input_ids"]))
            self.labels[-1][:question_len] = -100

            self.answers.append(row.A_Body)
            self.questions.append(row.Q_Body)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return (
            self.input_ids[idx],
            self.attn_masks[idx],
            self.labels[idx],
            self.answers[idx],
            self.questions[idx],
        )

In [19]:
train_dataset = Q_A_Dataset(train_df, tokenizer, max_length=max_length, tag="python")
test_dataset = Q_A_Dataset(test_df, tokenizer, max_length=max_length, tag="python")

In [20]:
del df

# Определим evaluation metrics

Определить метрики качества \ __BLEU, ROUGE__

Трекинг модели в wandb

API HF Trainer

In [24]:
# !pip install datasets
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=59fb2769ab08c05632a3b7acd45f6cb5a05189ec694e142484513ab96f071c1d
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [25]:
bleu = load_metric("bleu")
rouge = load_metric("rouge")

# Авторизумеся в wandb

In [50]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 23.6 MB/s 
     |████████████████████████████████| 181 kB 60.8 MB/s 
     |████████████████████████████████| 162 kB 65.7 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 158 kB 71.1 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 74.7 MB/s 
     |████████████████████████████████| 157 kB 67.6 MB/s 
     |████████████████████████████████| 157 kB 66.2 MB/s 
     |████████████████████████████████| 157 kB 72.1 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 71.8 MB/s 
     |████████████████████████████████| 156 kB 67.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f566b282a8e3ad3f70f35be264af918c09f4a07f5a86541dd6e6c2676c9

In [51]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [52]:
wandb.init(project="QA specific domain", entity="myashka")

wandb: Currently logged in as: myashka. Use `wandb login --relogin` to force relogin


In [53]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [54]:
%env WANDB_WATCH=all
%env WANDB_SILENT=true

env: WANDB_WATCH=all
env: WANDB_SILENT=true


# Определим Trainer и запустим обучение

In [67]:
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="wandb",  # enable logging to W&B
    run_name="gpt_neo_first_run",
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=lambda data: {
        "input_ids": torch.stack([f[0] for f in data]),
        "attention_mask": torch.stack([f[1] for f in data]),
        "labels": torch.stack([f[0] for f in data]),
    },
)

PyTorch: setting up devices


In [68]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1315
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


RuntimeError: ignored

# Evaluation

### #TODO
1. Дописать evaluate
2. Прогнать через модель dataset

In [28]:
text = "Question: What is the weather like?\nAnswer: Today is a good day!\n"
question = "Question: What is the weather like?\nAnswer: "

In [ ]:
def evaluate(model, test_dataset, tokenizer):
    model.eval()

    original_text, predicted_text, original_answer, predicted_answer = [], [], [], []

    bleu_scores = []
    rouge_scores = []

    for encoded_ids, _, _, answer, question in tqdm(test_dataset):

        original_text.append(f"Question: {question}\nAnswer: {answer}")
        original_answer.append(f"Answer: {answer}")

        question_len = len(f"Question: {question}\nAnswer: ")

        text_to_answer = f"<|startoftext|>Question: {question}\nAnswer:"

        enc_text_to_answer = tokenizer(text_to_answer, return_tensors="pt").input_ids

        generated_output = model.generate(
            enc_text_to_answer,
            do_sample=False,
            top_k=50,
            max_length=max_length,
            top_p=0.90,
            temperature=0,
            num_return_sequences=0,
        )
        # возвращается с pad
        generated_q_a = tokenizer.batch_decode(
            generated_output[0], skip_special_rokens=True
        )
        generated_a = generated_q_a[question_len:]

        predicted_text.append(generated_q_a)
        predicted_answer.append(generated_a)

        bleu_scores.append(
            bleu.compute(predictions=generated_a, references=answer)["bleu"]
        )
        rouge_scores.append(
            rouge.compute(predictions=[generated_a], references=[answer])[
                "rouge1"
            ].mid.fmeasure
        )

    return (
        original_text,
        predicted_text,
        original_answer,
        predicted_answer,
        np.mean(bleu_scores),
        np.mean(rouge_scores),
    )

In [ ]:
generated_q_a